In [10]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from pykrx import stock
import mplfinance as mpf
from tensorflow.keras.models import load_model
import os
import pandas as pd

patterns = ['ascending_triangle', 'descending_triangle', 'ascending_wedge', 'descending_wedge', 'double_top', 'double_bottom']
model_path = './save_model/chart_pattern_model.h5'

# 모델 불러오기
model = load_model(model_path)

In [11]:
# 예측을 위한 함수
def predict_pattern(model, ohlcv_data, window_size=70):
    """
    외부에서 입력받은 OHLCV 데이터를 기반으로 6개의 차트 패턴에 대한 확률을 예측하는 함수
    :param model: 학습된 CNN-LSTM 모델
    :param ohlcv_data: 새로운 OHLCV 데이터 (numpy 배열 형태로)
    :param window_size: 모델이 사용한 윈도우 크기 (슬라이딩 윈도우 크기)
    :return: 각 패턴에 속할 확률
    """
    # OHLCV 데이터를 정규화 (외부 데이터)
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(ohlcv_data)

    # 입력 데이터를 슬라이딩 윈도우로 변환
    if len(scaled_data) < window_size:
        raise ValueError(f"Input data length should be at least {window_size}")

    input_data = np.array([scaled_data[-window_size:]])  # 최신 데이터로 슬라이딩 윈도우 생성
    
    # 모델 예측 (확률 반환)
    predictions = model.predict(input_data)

    # 6개의 클래스에 대한 확률 출력
    return predictions[0]  # 예측된 확률

# DataFrame 분할 함수
def split_df(df, split_window):
    df_length = len(df)

    if df_length < split_window:
        print(f'{split_window} 기준으로 분할할 수 없습니다! 현재 길이: {df_length}')
    
    final_df_list = []

    for i in range(len(df) - split_window):
        df_start_point = i
        df_end_point = i + split_window
        splited_df = df.iloc[df_start_point:df_end_point]
        final_df_list.append(splited_df)

    return final_df_list    

In [20]:
ticker = '005930'
start_date = '20190101'
end_date = '20190601'

parnet_path = '../../../../store_data/raw/market_data/price/'
ticker_path = os.path.join(parnet_path, ticker)

start_date = pd.to_datetime(start_date).date()
end_date = pd.to_datetime(end_date).date()

time_delta = int(str(end_date - start_date).split(' ')[0])

if time_delta < 70:
    print('최소 70일 이상의 기간을 선택해야 합니다.')
elif time_delta <= 0:
    print('시작일이 종료일보다 빨라야합니다.')



'151'